In [18]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [2]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

In [3]:
ret, frame = cap.read()

In [4]:
img_bgr = cv2.resize(frame, (300, 300))

In [8]:
# tflite obj
!https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/raspberry_pi/download.sh

/bin/sh: https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/raspberry_pi/download.sh: No such file or directory


In [6]:
# python3
#
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Example using TF Lite to detect objects with the Raspberry Pi camera."""

# model
# https://www.tensorflow.org/lite/examples/object_detection/overview

# detect camera
# https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/raspberry_pi/detect_picamera.py

#open cv
# http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_gui/py_video_display/py_video_display.html

# ref
# https://qiita.com/kakinaguru_zo/items/c875ca7452c30a22289d#tensorflow-lite%E3%81%AE%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB%E3%81%AE%E6%94%B9%E9%80%A0%E7%89%88

#Quick start
# https://www.tensorflow.org/lite/guide/python?hl=ja

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import io
import re
import time

from annotation import Annotator

import numpy as np
# import picamera #for raspi

from PIL import Image
from tflite_runtime.interpreter import Interpreter

CAMERA_WIDTH = 640
CAMERA_HEIGHT = 480


def load_labels(path):
    """
    Loads the labels file. Supports files with or without index numbers.
    """
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        labels = {}
        for row_number, content in enumerate(lines):
            pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
            if len(pair) == 2 and pair[0].strip().isdigit():
                labels[int(pair[0])] = pair[1].strip()
            else:
                labels[row_number] = pair[0].strip()
    return labels


def set_input_tensor(interpreter, image):
    """　
    Sets the input tensor.
    """
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def get_output_tensor(interpreter, index):
    """
    Returns the output tensor at the given index.
    """
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


def detect_objects(interpreter, image, threshold):
    """
    Returns a list of detection results, each a dictionary of object info.
    """
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    # Get all output details
    boxes = get_output_tensor(interpreter, 0)
    classes = get_output_tensor(interpreter, 1)
    scores = get_output_tensor(interpreter, 2)
    count = int(get_output_tensor(interpreter, 3))

    results = []
    for i in range(count):
        if scores[i] >= threshold:
            result = {
              'bounding_box': boxes[i],
              'class_id': classes[i],
              'score': scores[i]
            }
            results.append(result)
    return results


def annotate_objects(annotator, results, labels):
    """
    Draws the bounding box and label for each object in the results.
    """
    for obj in results:
        # Convert the bounding box figures from relative coordinates
        # to absolute coordinates based on the original resolution
        ymin, xmin, ymax, xmax = obj['bounding_box']
        xmin = int(xmin * CAMERA_WIDTH)
        xmax = int(xmax * CAMERA_WIDTH)
        ymin = int(ymin * CAMERA_HEIGHT)
        ymax = int(ymax * CAMERA_HEIGHT)

        # Overlay the box, label, and score on the camera preview
        annotator.bounding_box([xmin, ymin, xmax, ymax])
        annotator.text([xmin, ymin],
                       '%s\n%.2f' % (labels[obj['class_id']], obj['score']))


In [2]:
!pip3 install https://dl.google.com/coral/python/tflite_runtime-2.1.0.post1-cp37-cp37m-macosx_10_14_x86_64.whl

     |████████████████████████████████| 1.2 MB 1.1 MB/s eta 0:00:01
You should consider upgrading via the '/Users/macico/.pyenv/versions/3.7.6/envs/py37b/bin/python3.7 -m pip install --upgrade pip' command.


In [7]:
parser = argparse.ArgumentParser(
  formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument(
  '--model', help='File path of .tflite file.', required=True)
parser.add_argument(
  '--labels', help='File path of labels file.', required=True)
parser.add_argument(
  '--threshold',
  help='Score threshold for detected objects.',
  required=False,
  type=float,
  default=0.4)
args = parser.parse_args(["--model","detect.tflite","--labels","coco_labels.txt"])

In [8]:
camera=None
# annotator = Annotator(camera)

labels = load_labels(args.labels)
interpreter = Interpreter(args.model)
interpreter.allocate_tensors()
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
#
image = cv2.resize(frame, (input_width, input_height))
start_time = time.monotonic()
results = detect_objects(interpreter, image, args.threshold)
elapsed_ms = (time.monotonic() - start_time) * 1000

# annotator.clear()
# annotate_objects(annotator, results, labels)
# annotator.text([5, 0], '%.1fms' % (elapsed_ms))
# annotator.update()

# stream.seek(0)
# stream.truncate()

In [9]:
image

array([[[240, 236, 241],
        [243, 238, 244],
        [240, 236, 241],
        ...,
        [ 45,  49,  55],
        [ 41,  46,  52],
        [ 42,  47,  53]],

       [[240, 236, 241],
        [239, 235, 240],
        [240, 236, 241],
        ...,
        [ 40,  45,  50],
        [ 42,  47,  53],
        [ 46,  51,  57]],

       [[237, 232, 237],
        [236, 231, 236],
        [240, 235, 240],
        ...,
        [ 40,  45,  51],
        [ 39,  43,  49],
        [ 46,  51,  57]],

       ...,

       [[111, 111, 114],
        [109, 108, 110],
        [106, 104, 107],
        ...,
        [ 70,  75,  80],
        [ 76,  82,  84],
        [ 79,  84,  88]],

       [[110, 110, 112],
        [107, 106, 108],
        [104, 102, 107],
        ...,
        [ 77,  82,  87],
        [ 71,  77,  80],
        [ 79,  84,  88]],

       [[ 97,  97,  99],
        [106, 105, 107],
        [105, 104, 108],
        ...,
        [ 78,  83,  89],
        [ 82,  87,  91],
        [ 78,  84,  87]]

In [19]:
results

[{'bounding_box': array([0.31420186, 0.07828394, 0.98990417, 0.7756872 ], dtype=float32),
  'class_id': 0.0,
  'score': 0.55078125},
 {'bounding_box': array([0.282165  , 0.24696724, 1.0033102 , 0.66265875], dtype=float32),
  'class_id': 16.0,
  'score': 0.5390625}]

In [17]:


def main():
    parser = argparse.ArgumentParser(
      formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument(
      '--model', help='File path of .tflite file.', required=True)
    parser.add_argument(
      '--labels', help='File path of labels file.', required=True)
    parser.add_argument(
      '--threshold',
      help='Score threshold for detected objects.',
      required=False,
      type=float,
      default=0.4)
    args = parser.parse_args()

    labels = load_labels(args.labels)
    interpreter = Interpreter(args.model)
    interpreter.allocate_tensors()
    _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
    #
    image = cv2.resize(frame, (input_width, input_height))
    start_time = time.monotonic()
    results = detect_objects(interpreter, image, args.threshold)
    elapsed_ms = (time.monotonic() - start_time) * 1000

    annotator.clear()
    annotate_objects(annotator, results, labels)
    annotator.text([5, 0], '%.1fms' % (elapsed_ms))
    annotator.update()

    stream.seek(0)
    stream.truncate()
    """
    with picamera.PiCamera(
        resolution=(CAMERA_WIDTH, CAMERA_HEIGHT), framerate=30) as camera:
        camera.start_preview()
    try:
        stream = io.BytesIO()
        annotator = Annotator(camera)
        for _ in camera.capture_continuous(
            stream, format='jpeg', use_video_port=True):
            stream.seek(0)
        image = Image.open(stream).convert('RGB').resize(
            (input_width, input_height), Image.ANTIALIAS)
        start_time = time.monotonic()
        results = detect_objects(interpreter, image, args.threshold)
        elapsed_ms = (time.monotonic() - start_time) * 1000

        annotator.clear()
        annotate_objects(annotator, results, labels)
        annotator.text([5, 0], '%.1fms' % (elapsed_ms))
        annotator.update()

        stream.seek(0)
        stream.truncate()

    finally:
        camera.stop_preview()
        
    """
    cam = cv2.VideoCapture(0)
    ret, img = cam.read()


if __name__ == '__main__':
    main()

NameError: name 'Annotator' is not defined